In [57]:
import os
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

n_glyphs = 6494

df1 = pd.read_csv('/Volumes/imvDrive/cfdb-django/glyphs-aligned-w-std_sign-images.csv', usecols=['std_sign', 'glyph_img'])
df2 = pd.read_csv('/Volumes/imvDrive/cfdb-django/glyph_count.csv', usecols=['std_sign', 'glyph_img']).sort_values(by='glyph_img')

start_file = '/Volumes/imvDrive/cfdb-django/media/start_file.csv'

if start_file in os.listdir('/Volumes/imvDrive/cfdb-django/media/'):
    os.remove(start_file)

f=open(start_file, 'a+')

df_group = df1.groupby(by=['std_sign'])
df_group = sorted(df_group, key=lambda x: len(x[1])) #https://stackoverflow.com/questions/22291395/sorting-the-grouped-data-as-per-group-size-in-pandas

for name,group in df_group:
    if  group.count()[0] >= 50:
        if(os.stat(start_file).st_size == 0):
            group.reset_index().to_csv(f)
        else:
            group.reset_index().to_csv(f, header=None)
        
df_usable = pd.read_csv(start_file)

'''
glyph_list = []
sign_list = []

##too computationally expensive to amplify list of data
##keep basic list and call _e, _s, _w pictures during model training

for index, row in df_usable.iterrows():
    if '.' in row['glyph_img']:
        split_glyph = row['glyph_img'].split('.')
        glyph_e = split_glyph[0]+'_e.'+split_glyph[1] 
        glyph_s = split_glyph[0]+'_s.'+split_glyph[1] 
        glyph_w = split_glyph[0]+'_w.'+split_glyph[1] 
        glyph_list.append(glyph_e)
        glyph_list.append(glyph_s)
        glyph_list.append(glyph_w)

        split_sign = row['std_sign'].split('.')
        sign_e = split_sign[0]+'_e.'+split_sign[1] 
        sign_s = split_sign[0]+'_s.'+split_sign[1] 
        sign_w = split_sign[0]+'_w.'+split_sign[1] 
        sign_list.append(sign_e)
        sign_list.append(sign_s)
        sign_list.append(sign_w)
print(glyph_list)
print(sign_list)
'''

df_shuffled = df_usable.sample(frac=1).reset_index(drop=True)
df_shuffled = df_shuffled.drop(['Unnamed: 0', 'index'], axis=1)

split_index0 = int(df_shuffled.count()[0]*0.75)
df_pre_testing = df_shuffled.iloc[:split_index0, :]
df_testing = df_shuffled.iloc[:split_index0, :]
df_testing.to_csv('/Volumes/imvDrive/cfdb-django/media/test_set/testing_data.csv')

split_index1 = int(df_pre_testing.count()[0]*0.95)
df_training = df_pre_testing.iloc[:split_index1, :]
df_validation = df_pre_testing.iloc[split_index1:, :]

df_training.to_csv('/Volumes/imvDrive/cfdb-django/media/training_set/training_data.csv')
df_validation.to_csv('/Volumes/imvDrive/cfdb-django/media/validation_set/validation_data.csv')

In [ ]:
#DATASTRUCTURES

###testing upload
###code including comments from: https://gist.github.com/eerwitt/518b0c9564e500b4b50f
filename_queue = tf.train.string_input_producer(
    tf.train.match_filenames_once("/Volumes/imvDrive/cfdb-django/media/training_set/training_data/*.jpeg"))

# Read an entire image file which is required since they're JPEGs, if the images
# are too large they could be split in advance to smaller files or use the Fixed
# reader to split up the file.
image_reader = tf.WholeFileReader()

# Read a whole file from the queue, the first returned value in the tuple is the
# filename which we are ignoring.
_, image_file = image_reader.read(filename_queue)

# Decode the image as a JPEG file, this will turn it into a Tensor which we can
# then use in training.
image = tf.image.decode_jpeg(image_file)

# Start a new session to show example output.
with tf.Session() as sess:
    # Required to get the filename matching to run.
    tf.initialize_all_variables().run()

    # Coordinate the loading of image files.
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)

    # Get an image tensor and print its value.
    image_tensor = sess.run([image])
    print(image_tensor)

    # Finish off the filename queue coordinator.
    coord.request_stop()
coord.join(threads)